In [1]:
con = mysql.connector.connect(user="pc2_grupo5", password = "Proyectos.23", host = "195.235.211.197")

NameError: name 'mysql' is not defined

In [ ]:
cursor = con.cursor()
cursor.execute("USE pc2_grupo5;")

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import mysql
import mysql.connector

con = mysql.connector.connect(user="pc2_grupo5", password = "Proyectos.23", host = "195.235.211.197")
cursor = con.cursor()
cursor.execute("USE pc2_grupo5;")

body = ["chest","shoulders","traps","biceps","forearms","obliques","abdominals","quads","calves","lats","lowerback","glutes","hamstrings","traps_middle"]

for musculo in body:
    cursor.execute("INSERT INTO muscles (name) VALUES ('" + musculo + "')")
    con.commit()


In [37]:
import requests
from bs4 import BeautifulSoup
import json
import mysql.connector
import datetime

# Conectarse a la base de datos
mydb = mysql.connector.connect(
    host="195.235.211.197",
    user="pc2_grupo5",
    password="Proyectos.23",
    database="pc2_grupo5"
)

# Crear un cursor para ejecutar las consultas SQL
cursor = mydb.cursor()


ejercicios = []
pasosdelejercicios = []
nombredelejercicio = []

    
msc = {}


body = ["chest","shoulders","traps","biceps","forearms","obliques","abdominals","quads","calves","lats","lowerback","glutes","hamstrings","traps_middle"]
bodyt=["chest"]

timestamp = datetime.datetime.now()

for mus in body:
    musculo_verificar = mus
    print(musculo_verificar)
    
    cursor.execute("SELECT id FROM muscles WHERE name = %s", (musculo_verificar,))
    result = cursor.fetchone()
    
    # Si el músculo no existe, insertarlo en la tabla "muscle"
    if not result:
        cursor.execute("INSERT INTO muscles (name, created_at) VALUES (%s, %s)", (musculo_verificar,timestamp))
        muscle_id = cursor.lastrowid
    else:
        muscle_id = result[0]  
    
    # URL de la página web a scrappear
    url = 'https://musclewiki.com/exercises/male/' + str(mus)

    # Realizar la petición HTTP y obtener el contenido de la página web
    response = requests.get(url)
    html = response.content

    # Crear el objeto BeautifulSoup y analizar el contenido HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Buscar los elementos que contienen los ejercicios por músculo
    msc[mus] = []
    
    muscles = soup.find_all('h3')
    descript = soup.find_all('ol')
    difficult = soup.find_all('p')
    videos = soup.find_all('video')
    
    videoss = []
    for video in videos:
        url = video['src']
        videoss.append(url)
    
    difficulties=[]
    for dif in difficult:
        d = dif.text.split()[1]
        difficulties.append(d)

    steps = []
    for desc in descript:
        descripciones = desc.find_all('li')
        muscle_steps = []  # Lista para almacenar los pasos del ejercicio actual
        for d in descripciones:
            muscle_steps.append(d.text)
        steps.append(muscle_steps)  # Agregar los pasos del ejercicio actual a la lista de pasos por músculo
    
    step_div = []
    for step in steps:
        step_div.append(step)
    
    i=0
    j=0
    
    
    
    # Recorrer cada músculo encontrado e imprimir los ejercicios correspondientes
    for muscle in muscles:
        # Buscar los enlaces de los ejercicios correspondientes a este músculo
        exercises = muscle.find_all('span',{'class':"mw-headline"})
        # Imprimir los enlaces de los ejercicios
        for exercise in exercises:

            ej = exercise.text.replace('\n', '').replace('Copied', '').strip()
             # Buscar si el ejercicio ya existe en la tabla "exercise" para el músculo actual
            cursor.execute("SELECT id FROM exercises WHERE name = %s AND muscle_id = %s", (ej, muscle_id))
            result = cursor.fetchone()
            

            
             # Si el ejercicio no existe, insertarlo en la tabla "exercise"
            if not result:
                cursor.execute("INSERT INTO exercises (name, difficulty, url, muscle_id, created_at) VALUES (%s, %s, %s, %s, %s)", 
                               (ej, difficulties[i], videoss[j], muscle_id, timestamp))
                exercise_id = cursor.lastrowid
            else:
                exercise_id = result[0]
            
            description = '\n'.join(step_div[i])
            #print(str(len(description)) + " - exercise_id:" + str(exercise_id))
            cursor.execute("INSERT INTO steps (description, exercise_id) VALUES (%s, %s)", (description, exercise_id))
            
            i += 1

#primero hacer un select del musculo a insertar (coger el id)

#luego hacer un select de todos los ejercicios que tiene  ese musclulo
    # si ya existe ese ejercico no lo inserto
    # si no existe dicho ejercicio lo inserto

#finalmente del ejercico hacer otro select para coger el id e insertar todos los pasos 

# Hacer commit de las transacciones y cerrar la conexión
mydb.commit()
cursor.close()
mydb.close()

# Iterar sobre los músculos y los ejercicios encontrados


chest
165 - exercise_id:113
268 - exercise_id:114
258 - exercise_id:115
294 - exercise_id:116
shoulders
422 - exercise_id:117
208 - exercise_id:118
261 - exercise_id:119
146 - exercise_id:120
traps
221 - exercise_id:121
98 - exercise_id:122
292 - exercise_id:123
146 - exercise_id:124
biceps
340 - exercise_id:125
216 - exercise_id:126
216 - exercise_id:127
151 - exercise_id:128
forearms
134 - exercise_id:129
216 - exercise_id:130
188 - exercise_id:131
214 - exercise_id:132
obliques
272 - exercise_id:133
144 - exercise_id:134
216 - exercise_id:135
172 - exercise_id:136
abdominals
142 - exercise_id:137
443 - exercise_id:138
208 - exercise_id:139
170 - exercise_id:140
quads
474 - exercise_id:141
260 - exercise_id:142
316 - exercise_id:143
245 - exercise_id:144
calves
360 - exercise_id:145
210 - exercise_id:146
436 - exercise_id:147
279 - exercise_id:148
lats
216 - exercise_id:149
166 - exercise_id:150
491 - exercise_id:151
216 - exercise_id:152
lowerback
326 - exercise_id:153
228 - exercis

In [ ]:

# Iterar sobre los músculos y los ejercicios encontrados
for mus, ejercicios in json.items():
    # Buscar el ID del músculo actual en la tabla "muscle"

    cursor.execute("SELECT id FROM muscles WHERE name = %s", (mus))
    result = cursor.fetchone()
    
    # Si el músculo no existe, insertarlo en la tabla "muscle"
    if not result:
        cursor.execute("INSERT INTO muscles (name) VALUES (%s)", (mus))
        muscle_id = cursor.lastrowid
    else:
        muscle_id = result[0]
    
    # Iterar sobre los ejercicios y sus pasos
    for ejercicio in ejercicios:
        # Buscar si el ejercicio ya existe en la tabla "exercise" para el músculo actual
        cursor.execute("SELECT id FROM exercises WHERE name = %s AND muscle_id = %s", (ejercicio['name'], muscle_id))
        result = cursor.fetchone()
        
        # Si el ejercicio no existe, insertarlo en la tabla "exercise"
        if not result:
            cursor.execute("INSERT INTO exercises (name, difficulty, url, muscle_id) VALUES (%s, %s, %s, %s)", 
                           (ejercicio['name'], ejercicio['difficulty'],ejercicio['video1'], muscle_id))
            exercise_id = cursor.lastrowid
        else:
            exercise_id = result[0]
        
        # Iterar sobre los pasos del ejercicio y agregarlos a la tabla "steps"
        for i, step in enumerate(ejercicio['steps']):
            cursor.execute("INSERT INTO steps (description, exercise_id) VALUES (%s, %s)", 
                           (step, exercise_id))

# Hacer commit de las transacciones y cerrar la conexión
mydb.commit()
cursor.close()
mydb.close()
